In [1]:
import os
import datetime
import pandas as pd
import numpy as np
import glob

#表示行数の増加
pd.set_option('display.max_rows', 500)

today = datetime.datetime.now()

y = today.year
m = today.month

ym = str(y) + str('{0:02d}'.format(m))

#サブブランド単位で集計するショップを指定
subbrandUnit = \
["PALEMOBA ONLINESTORE",]

#大カテゴリにて集計するショップ
category_L = \
["PALEMOBA ONLINESTORE",]

#入庫集計対象のショップ（サブブランド単位）
receiveSubbrand = \
["PALEMOBA事業部", ]

receiveSubbrand_convert = \
{"PALEMOBA事業部":"PALEMOBA ONLINESTORE"}

#暫定でのroot dir
root = "/Users/miyamoto/Documents/Root/raw/monthly_agg_sub"

In [2]:
#共通関数群

def file_ditect(key_dir):#指定ディレクトリに存在するファイルパスの一覧取得
    path = root + "/" + key_dir + "/" + ym
    
    files = []

    # パスで指定したファイルの一覧をリスト形式で取得
    files = glob.glob(path + '/*.csv')

    #print(files)

    for file in files:
        print(file)
    
    return files
        
def palemoba_load(pm):#月の調整
    #pm = pd.read_csv(file)
    
    pm_monthPlus = pm.query('日 >= 21')
    pm_monthPlus.loc[:, "月"] += 1
    pm_over = pm_monthPlus.query('月 == 13')
    pm_over.loc[:, "年"] += 1
    pm_over.loc[:, "月"] -= 12
    pm_monthPlus_noChange = pm_monthPlus.query('月 != 13')
    pm_noChange = pm.query('日 < 21')
    
    pm = pd.concat([pm_over, pm_monthPlus_noChange, pm_noChange], axis = 0)
    
    return pm

def import_csv(df, file):
    
    df = palemoba_load(df)
    df = df[~df["ブランド名"].str.contains("ノベルティ")]
    
    return df

In [3]:
#sell用の関数群

def import_sell(file):
    dataframe = pd.read_csv(file)
    dataframe["年"] = pd.to_datetime(dataframe["日付"]).dt.year
    dataframe["月"] = pd.to_datetime(dataframe["日付"]).dt.month
    dataframe["日"] = pd.to_datetime(dataframe["日付"]).dt.day
    df = import_csv(dataframe, file)
    
    return df

def subbrand_sell(df):
    agg_subbrand = df.query("ショップ名 in @subbrandUnit")
    
    if agg_subbrand.empty:
        return

    agg_subbrand = agg_subbrand.groupby(["年", "月", "ショップ名", "ブランド名", "サブブランド名"]).agg({'出荷金額':'sum','返品金額':'sum','出荷金額-返品金額':'sum'})

    agg_subbrand.rename(columns={'出荷金額-返品金額': '売上金額'}, inplace=True)
    
    agg_subbrand = agg_subbrand.stack().reset_index()

    return agg_subbrand


In [4]:
#sell処理
files_sell = file_ditect("sell")
sell_list_subbrand = []

for file in files_sell:
    sell = import_sell(file)
    df_subbrand_sell = subbrand_sell(sell)
    sell_list_subbrand.append(df_subbrand_sell)

/Users/miyamoto/Documents/Root/raw/monthly_agg_sub/sell/202101/売上（出荷-顧客返品）_20210121094338_PALEMOBA.csv


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [5]:
sb_sell = pd.concat(sell_list_subbrand,axis = 0).reset_index()
sb_sell = sb_sell[sb_sell.columns[sb_sell.columns != 'index']]
sb_sell.rename(columns={"level_5":"属性", 0:"scalar"}, inplace =True)

In [6]:
sb_sell

,年,月,ショップ名,ブランド名,サブブランド名,属性,scalar
0,2021,1,PALEMOBA ONLINESTORE,Ludic Park,< NULL >,出荷金額,7700
1,2021,1,PALEMOBA ONLINESTORE,Ludic Park,< NULL >,返品金額,0
2,2021,1,PALEMOBA ONLINESTORE,Ludic Park,< NULL >,売上金額,7700
3,2021,1,PALEMOBA ONLINESTORE,Ludic Park,Daisy Merry Cool,出荷金額,526692
4,2021,1,PALEMOBA ONLINESTORE,Ludic Park,Daisy Merry Cool,返品金額,0
5,2021,1,PALEMOBA ONLINESTORE,Ludic Park,Daisy Merry Cool,売上金額,526692
6,2021,1,PALEMOBA ONLINESTORE,Ludic Park,Daisy Merry Girly,出荷金額,2519457
7,2021,1,PALEMOBA ONLINESTORE,Ludic Park,Daisy Merry Girly,返品金額,41810
8,2021,1,PALEMOBA ONLINESTORE,Ludic Park,Daisy Merry Girly,売上金額,2477647
9,2021,1,PALEMOBA ONLINESTORE,Ludic Park,Lilou de chouchou,出荷金額,560149


In [7]:
#//sell処理

In [8]:
#order用の関数群
def import_order(file):
    df = pd.read_csv(file)    
    
    shopList = df["ショップ名"].unique()
    
    for shop in shopList:
        df_query = "ショップ名 == @shop"
        target = df.query(df_query)
        df.loc[target.index, "販売ステータス"] = \
        df.loc[target.index, "販売ステータス"].replace({"セール（シークレット）":"セール", 'アウトレット':"セール", '予約（シークレット）':"予約"})
    
    df["年"] = pd.to_datetime(df["受注日"]).dt.year
    df["月"] = pd.to_datetime(df["受注日"]).dt.month
    df["日"] = pd.to_datetime(df["受注日"]).dt.day
    df["大カテゴリ名"] = "受注金額/大カテゴリ/" + df["大カテゴリ名"]
    df["中カテゴリ名"] = "受注金額/中カテゴリ/" + df["中カテゴリ名"]
    df["販売ステータス"] = "受注金額/販売ステータス/" + df["販売ステータス"]
    df["購入ステータス"] = "受注件数/購入ステータス/" + df["購入ステータス"]
    
    
    df = import_csv(df, file)
    
    return df

def subbrand_order(df):
    df = df.query("ショップ名 in @subbrandUnit")
    
    if df.empty:
        return
    
    agg = df.query("受注金額 != 0")
    agg = df.groupby(["年", "月", "ショップ名", "ブランド名", "サブブランド名"]).agg({'受注番号':'nunique','受注数':'sum', '受注金額':'sum'})
    agg.rename(columns={'受注番号': '受注件数'}, inplace=True)
    agg = agg.stack()
    
    agg_site = df.query("受注金額 != 0")
    agg_site = agg_site.groupby(["年", "月", "ショップ名"]).agg({'受注番号':'nunique','受注数':'sum'})
    agg_site.rename(columns={'受注番号': '受注件数'}, inplace=True)
    agg_site = agg_site.stack()
    
    agg_sellStatus = df.groupby(["年", "月", "ショップ名", "ブランド名", "サブブランド名", "販売ステータス"]).agg({'受注金額':'sum'})
    agg_sellStatus = agg_sellStatus.stack()
    
    agg_category_L = df.groupby(["年", "月", "ショップ名", "ブランド名", "サブブランド名", "大カテゴリ名"]).agg({'受注金額':'sum'})
    agg_category_L = agg_category_L.stack()
    
    agg_purStatus = df.query("受注金額 != 0")
    agg_purStatus = agg_purStatus.groupby(["年", "月", "ショップ名", "購入ステータス"]).agg({'受注番号':'nunique'})
    agg_purStatus.rename(columns={'受注番号': '受注件数'}, inplace=True)
    agg_purStatus = agg_purStatus.stack()
    
    df_agg = pd.concat([agg, agg_sellStatus, agg_category_L], axis = 0)
    df_agg = df_agg.reset_index()
    df_agg.rename(columns={'level_5': '属性'}, inplace=True)
    
    df_agg_shop = pd.concat([agg_site,  agg_purStatus], axis = 0)
    df_agg_shop = df_agg_shop.reset_index()
    df_agg_shop.rename(columns={'level_3': '属性'}, inplace=True)
    
    agg_total = pd.concat([df_agg, df_agg_shop], axis = 0)
    
    return agg_total

In [9]:
#order処理
files_order = file_ditect("order")

order_list_subbrand = []

for file in files_order:
    order = import_order(file)
    df_subbrand_order = subbrand_order(order)
    order_list_subbrand.append(df_subbrand_order)

/Users/miyamoto/Documents/Root/raw/monthly_agg_sub/order/202101/受注_20210121093833_PALEMOBA.csv


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (7,9,27,30,33,36) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:61: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [10]:
sb_order = pd.concat(order_list_subbrand,axis = 0).reset_index()
sb_order = sb_order[sb_order.columns[sb_order.columns != 'index']]
sb_order.rename(columns={"level_5":"属性", 0:"scalar"}, inplace =True)
sb_order = sb_order[["年", "月", "ショップ名", "ブランド名", "サブブランド名", "属性", "scalar"]]

In [11]:
sb_order

,年,月,ショップ名,ブランド名,サブブランド名,属性,scalar
0,2021,1,PALEMOBA ONLINESTORE,Ludic Park,< NULL >,受注件数,4
1,2021,1,PALEMOBA ONLINESTORE,Ludic Park,< NULL >,受注数,3
2,2021,1,PALEMOBA ONLINESTORE,Ludic Park,< NULL >,受注金額,7700
3,2021,1,PALEMOBA ONLINESTORE,Ludic Park,Daisy Merry Cool,受注件数,161
4,2021,1,PALEMOBA ONLINESTORE,Ludic Park,Daisy Merry Cool,受注数,258
5,2021,1,PALEMOBA ONLINESTORE,Ludic Park,Daisy Merry Cool,受注金額,530202
6,2021,1,PALEMOBA ONLINESTORE,Ludic Park,Daisy Merry Girly,受注件数,501
7,2021,1,PALEMOBA ONLINESTORE,Ludic Park,Daisy Merry Girly,受注数,922
8,2021,1,PALEMOBA ONLINESTORE,Ludic Park,Daisy Merry Girly,受注金額,2557626
9,2021,1,PALEMOBA ONLINESTORE,Ludic Park,Lilou de chouchou,受注件数,209


In [12]:
#//order処理

In [13]:
#newProduct関数群
def import_newProduct(file):
    dataframe = pd.read_csv(file)
    df = import_csv(dataframe, file)
    return df

def subbrand_newProduct(df):
    df = df.query("ショップ名 in @subbrandUnit")
    
    if df.empty:
        return
    
    agg = df.groupby(["年", "月", "ショップ名", "ブランド名"]).agg({'品番数':'sum','日':'nunique'})
    agg.rename(columns={'品番数':'新規アップ数', '日': 'アップ日数'}, inplace=True)
    agg = agg.stack()

    return agg


In [14]:
#newProduct処理
files_newProduct = file_ditect("newProduct")

newProduct_list_subbrand = []

for file in files_newProduct:
    newProduct = import_newProduct(file)
    df_subbrand_newProduct = subbrand_newProduct(newProduct)
    newProduct_list_subbrand.append(df_subbrand_newProduct)

/Users/miyamoto/Documents/Root/raw/monthly_agg_sub/newProduct/202101/新規商品アップ数_20210121093622_PALEMOBA.csv


In [15]:
sb_newProduct = pd.concat(newProduct_list_subbrand,axis = 0).reset_index()
sb_newProduct = sb_newProduct[sb_newProduct.columns[sb_newProduct.columns != 'index']]
sb_newProduct.rename(columns={"level_4":"属性", 0:"scalar"}, inplace =True)

In [16]:
sb_newProduct

,年,月,ショップ名,ブランド名,属性,scalar
0,2021,1,PALEMOBA ONLINESTORE,Ludic Park,新規アップ数,96
1,2021,1,PALEMOBA ONLINESTORE,Ludic Park,アップ日数,9
2,2021,1,PALEMOBA ONLINESTORE,Re-J＆SUPURE,新規アップ数,103
3,2021,1,PALEMOBA ONLINESTORE,Re-J＆SUPURE,アップ日数,12


In [17]:
#request関数群
def import_request(file):
    df = pd.read_csv(file)
    
    df["リクエスト金額"] = df["定価"]*df["登録数"]
    
    df["年"] = pd.to_datetime(df["対象日"]).dt.year
    df["月"] = pd.to_datetime(df["対象日"]).dt.month
    df["日"] = pd.to_datetime(df["対象日"]).dt.day
    
    df = df.query("リクエストタイプ == '再入荷' ")
    
    df = import_csv(df, file)
    
    return df

def subbrand_request(df):
    df = df.query("ショップ名 in @subbrandUnit")
    
    if df.empty:
        return
    
    df_reg = df.query("登録数 != 0")
    df_send = df.query("送信数 != 0")
    
    agg_reg = df_reg.groupby(["年", "月", "ショップ名", "ブランド名"]).agg({'商品コード':'nunique','リクエスト金額':'sum'})
    agg_reg.rename(columns={'商品コード': '登録品番数'}, inplace=True)
    
    agg_send = df_send.groupby(["年", "月", "ショップ名", "ブランド名"]).agg({'商品コード':'nunique'})
    agg_send.rename(columns={'商品コード': '送信品番数'}, inplace=True)
    
    agg = pd.merge(agg_reg, agg_send, on=["年", "月", "ショップ名", "ブランド名"], how='left')
    
    agg = agg.stack()

    return agg


In [18]:
#request処理
files_request = file_ditect("request")

/Users/miyamoto/Documents/Root/raw/monthly_agg_sub/request/202101/リクエストメール集計_20210121094236_PALEMOBA.csv


In [19]:
request_list_subbrand = []

for file in files_request:
    request = import_request(file)
    df_subbrand_request = subbrand_request(request)
    request_list_subbrand.append(df_subbrand_request)

In [20]:
sb_request = pd.concat(request_list_subbrand,axis = 0).reset_index()
sb_request = sb_request[sb_request.columns[sb_request.columns != 'index']]
sb_request.rename(columns={"level_4":"属性", 0:"scalar"}, inplace =True)

In [21]:
sb_request

,年,月,ショップ名,ブランド名,属性,scalar
0,2021,1,PALEMOBA ONLINESTORE,Ludic Park,登録品番数,183.0
1,2021,1,PALEMOBA ONLINESTORE,Ludic Park,リクエスト金額,1707120.0
2,2021,1,PALEMOBA ONLINESTORE,Ludic Park,送信品番数,47.0
3,2021,1,PALEMOBA ONLINESTORE,Re-J＆SUPURE,登録品番数,372.0
4,2021,1,PALEMOBA ONLINESTORE,Re-J＆SUPURE,リクエスト金額,10667619.0
5,2021,1,PALEMOBA ONLINESTORE,Re-J＆SUPURE,送信品番数,102.0


In [22]:
#receive関数群
def import_receive(file):
    df = pd.read_csv(file)
    
    df["年"] = pd.to_datetime(df["入庫実施日"]).dt.year
    df["月"] = pd.to_datetime(df["入庫実施日"]).dt.month
    df["日"] = pd.to_datetime(df["入庫実施日"]).dt.day
    
    df = import_csv(df, file)
    
    df = df[~df["ブランド名"].str.contains("ノベルティ")]
    
    return df

def subbrand_receive(df):
    df = df.query("事業部 in @receiveSubbrand")
    df = df.replace(receiveSubbrand_convert)
    
    if df.empty:
        return
    
    agg = df.groupby(["年", "月", "事業部", "ブランド名"]).agg({'入庫実績':'sum'})
    agg.rename(columns={'入庫実績':'入庫金額'}, inplace=True)
    
    agg = agg.stack()

    return agg


In [23]:
#receive処理
files_receive = file_ditect("receive")

/Users/miyamoto/Documents/Root/raw/monthly_agg_sub/receive/202101/入庫実績_20210121093944_PALEMOBA.csv


In [24]:
receive_list_subbrand = []

for file in files_receive:
    receive = import_receive(file)
    df_subbrand_receive = subbrand_receive(receive)
    receive_list_subbrand.append(df_subbrand_receive)

In [25]:
sb_receive = pd.concat(receive_list_subbrand,axis = 0).reset_index()
sb_receive = sb_receive[sb_receive.columns[sb_receive.columns != 'index']]
sb_receive.rename(columns={"level_4":"属性", 0:"scalar","事業部":"ショップ名"}, inplace =True)

In [26]:
sb_receive

,年,月,ショップ名,ブランド名,属性,scalar
0,2021,1,PALEMOBA ONLINESTORE,Ludic Park,入庫金額,18184089
1,2021,1,PALEMOBA ONLINESTORE,Re-J＆SUPURE,入庫金額,37453617


In [27]:
sb_total = pd.concat([sb_sell, sb_order, sb_newProduct, sb_request, sb_receive], axis = 0)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [28]:
sb_total = sb_total[["年", "月", "ショップ名", "ブランド名", "サブブランド名", "属性", "scalar"]].reset_index()
sb_total = sb_total[sb_total.columns[sb_total.columns != 'index']]

In [29]:
sb_total

,年,月,ショップ名,ブランド名,サブブランド名,属性,scalar
0,2021,1,PALEMOBA ONLINESTORE,Ludic Park,< NULL >,出荷金額,7700.0
1,2021,1,PALEMOBA ONLINESTORE,Ludic Park,< NULL >,返品金額,0.0
2,2021,1,PALEMOBA ONLINESTORE,Ludic Park,< NULL >,売上金額,7700.0
3,2021,1,PALEMOBA ONLINESTORE,Ludic Park,Daisy Merry Cool,出荷金額,526692.0
4,2021,1,PALEMOBA ONLINESTORE,Ludic Park,Daisy Merry Cool,返品金額,0.0
5,2021,1,PALEMOBA ONLINESTORE,Ludic Park,Daisy Merry Cool,売上金額,526692.0
6,2021,1,PALEMOBA ONLINESTORE,Ludic Park,Daisy Merry Girly,出荷金額,2519457.0
7,2021,1,PALEMOBA ONLINESTORE,Ludic Park,Daisy Merry Girly,返品金額,41810.0
8,2021,1,PALEMOBA ONLINESTORE,Ludic Park,Daisy Merry Girly,売上金額,2477647.0
9,2021,1,PALEMOBA ONLINESTORE,Ludic Park,Lilou de chouchou,出荷金額,560149.0


In [30]:
sb_total.to_csv('PALEMOBA ONLINESTORE_月次レビュー.csv', encoding='utf_8_sig')